# About this Notebook

Few weeks ago I shared code for Siamese Style Training Strategy with contrastive loss [here](https://www.kaggle.com/tanulsingh077/siamese-style-training-efficient-net-b0) and the dataset used was published [here](https://www.kaggle.com/tanulsingh077/shopee-siamese-training).

This is the helper code to generate data for Siamese Style Training with Contrastive Loss . @xhlulu has been kind enough to share code to generate data for Siamese Style training with Triplet Loss [here](https://www.kaggle.com/xhlulu/shopee-generate-data-for-triplet-loss)

Now we all can train models with these two different losses and see what works for us . I will also upload the inference kernels as soon as I get promising results by any of these approaches

The Logic used in the following code is almost similar as xhlulu's only it differs in the way I extract titles . The one thing which I feel the data suffers from is the number of positive examples since the negative pairs will be much larger than the positive pairs for products having just 2 items

Please let me know in the comments if you don't understand the code ,I will update this notebook with line by line explanantion then

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
data = pd.read_csv('../input/shopee-product-matching/train.csv')
data_without_dup = data.drop_duplicates(subset='label_group')

In [ ]:
posting_id_dict = data.groupby('label_group')['posting_id'].unique().to_dict()
title_dict = data.groupby('label_group')['title'].unique().to_dict()
image_dict = data.groupby('label_group')['image'].unique().to_dict()

In [ ]:
label_groups = data_without_dup.label_group.values.tolist()

In [ ]:
new_data = []
for i,label_group in tqdm(enumerate(data_without_dup['label_group'])):
    matches = posting_id_dict[label_group].tolist()
    titles = title_dict[label_group].tolist()
    images = image_dict[label_group].tolist()
    
    index = np.random.randint(2,len(label_groups)-2)
    while (index== i):
        index = np.random.randint(0,len(label_groups))
    
    if len(matches) == 2:
        if len(titles) == 2:
            matches.extend(titles)
            matches.extend(images)
            matches.extend([1])
        else:
            matches.extend([titles[0],titles[0]])
            matches.extend(images)
            matches.extend([1])
        new_data.append(matches)
        new_data.append([matches[0],posting_id_dict[label_groups[index]][0],titles[0],title_dict[label_groups[index]][0],images[0],image_dict[label_groups[index]][0],0])
        new_data.append([matches[0],posting_id_dict[label_groups[index+1]][0],titles[0],title_dict[label_groups[index+1]][0],images[0],image_dict[label_groups[index+1]][0],0])
        new_data.append([matches[0],posting_id_dict[label_groups[index-1]][0],titles[0],title_dict[label_groups[index-1]][0],images[0],image_dict[label_groups[index-1]][0],0])
    
    else:
        for match,title,image in zip(matches[1:],titles[1:],images[1:]):
            new_data.append([matches[0],match,titles[0],title,images[0],image,1])
            new_data.append([matches[0],posting_id_dict[label_groups[index]][0],titles[0],title_dict[label_groups[index]][0],images[0],image_dict[label_groups[index]][0],0])

In [ ]:
siamese_data = pd.DataFrame(new_data,columns=['posting_id_1','posting_id_2','title_1','title_2','image_1','image_2','label'])

In [ ]:
siamese_data

In [ ]:
siamese_data.to_csv('siamese_data.csv',index=False)